In [1]:
import cv2
import numpy as np

In [2]:
def order_points(pts):
    #Создаем заглушку
    rect = np.zeros((4, 2), dtype = "float32")
    pts_sum = pts.sum(axis=1)

    #rect[0] - левый верхний угол, rect[2] - правый нижний
    rect[0] = pts[np.argmin(pts_sum)]
    rect[2] = pts[np.argmax(pts_sum)]

    pts_diff = np.diff(pts, axis = 1)

    #rect[1] - правый верхний угол, rect[3] - левый нижний
    rect[1]= pts[np.argmin(pts_diff)]
    rect[3]= pts[np.argmax(pts_diff)]

    return rect



In [3]:
def four_point_transform(orig_img, img_points):
    #Разбиваем четырехугольник на точки
    tl,tr,br,bl = img_points
    #Рассчет длинны векторов ширины и высоты каждой из сторон 4-х угольника
    top_width = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    bottom_width = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    left_height = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    right_height = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))

    #Берем максимальную высоту и ширину для дальнейшего преобразования изображения
    max_width = max(int(top_width), int(bottom_width))
    max_height = max(int(left_height), int(right_height))

    #Массив с координатами точек в порядке: левый верхний угол, правый верхний, правый нижний, левый нижний.
    #Отправная точка - левый верхний угол
    dest_matrix = np.array([
    [0, 0],
    [max_width - 1, 0],
    [max_width - 1, max_height - 1],
    [0, max_height - 1]], dtype = "float32")

    #Собираем матрицу трансформации
    transf_matrix = cv2.getPerspectiveTransform(img_points, dest_matrix)

    #Применяем трансформацию
    transformed_img = cv2.warpPerspective(orig_img, transf_matrix, (max_width, max_height))

    #Возвращаем трансформированное изображение
    return transformed_img